<a href="https://colab.research.google.com/github/RiverGumSecurity/AILabs/blob/main/022_LargeLanguageModels/SFT_bhisblogs.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import warnings
import os
import sys
import pathlib
try:
    import unsloth
except:
    if 'google.colab' in sys.modules:
        !pip install unsloth

# suppresses some noisey warnings which are just annoying
warnings.filterwarnings('ignore')
max_seq_length = 4096

# Setup Hugging Face Credentials.
HF_APIKEY = ''
if 'google.colab' in sys.modules:
    from google.colab import userdata
    HF_APIKEY = userdata.get('HF_APIKEY')
else:
    with open(pathlib.Path.home() / '.hfkey') as hf:
        HF_APIKEY = hf.read().strip()
if not HF_APIKEY:
    print('[-] ERROR: Cannot continue without Hugging Face API Key')
    sys.exit(0)
os.environ['HF_TOKEN'] = HF_APIKEY

model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = None, load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
import re
import json
import datasets
import requests
from bs4 import BeautifulSoup

# get dataset
r = requests.get('https://raw.githubusercontent.com/RiverGumSecurity/Datasets/refs/heads/main/BHIS/bhis-blogs-cleaned.json')
data = json.loads(r.text)
ds = datasets.Dataset.from_list(data)

In [12]:
from transformers import TextStreamer

prompt = """
### Instruction:
Based on the following extract, generate five instruction-answer pairs. Each instruction
must ask to write about a specific topic contained in the context.
Each answer must provide a relevant paragraph based on the information found in the
context.
Only use concepts from the context to generate the instructions.
Instructions must never explicitly mention a context, a system, a course, or an extract.
Instructions must be self-contained and general.
Answers must imitate the writing style of the context.

Provide your response in JSON format with the following structure:

{{
"instruction_answer_pairs": [
        {{"instruction": "...", "answer": "..."}},
    ]
}}

### Input:
{}

### Response:
"""

unsloth.FastLanguageModel.for_inference(model)

for b in ds:
    inputs = tokenizer([prompt.format(b['output'])], return_tensors = "pt").to("cuda")
    tensor = model.generate(**inputs, max_new_tokens = 2048)
    output = tokenizer.decode(tensor[0], skip_special_tokens=True)
    print(output.split('### Response:')[1])


```json
{
  "instruction_answer_pairs": [
    {
      "instruction": "Write about the graph",
      "answer": "Mick Douglas Take look at this chart from last year's Verizon Data Breach Report It shows who notified the breached party when they were attacked This graph is a sad indictment for all of us in the information protection industry This chart means that only about 1 in 8 times there's a breach it's found out by an internal party Of course this means that 7 times of those 8 it's discovered by someone external to your organization That has got to hurt If you doubt me just ask the PR firm or department of any breached organization There's two ways you can look at this chart First you can despair that all the spending on IT security has largely been what Wasted We'll not go that far Misallocated might be a better way of thinking about how your funds were spent Secondly we can't help but see that there's so much room for improvement that virtually anything we change could make a big

KeyboardInterrupt: 